# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [82]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [83]:
#Create dataframe
cities_df = pd.read_csv('../Resources/cities.csv')
cities_df.dropna(inplace = True) 
cities_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [84]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [85]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)

In [86]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [90]:
# Desired weather conditions: Temperature above 50 degrees, Wind less than 15mph & Humidity less than 60%
# Add Desired Temp column
temp_cond = cities_df[cities_df["Max Temp"] > 65]
first_combined_cond = pd.merge(cities_df, temp_cond, how='outer', on='City')
first_combined_cond = first_combined_cond.rename(columns={"Max Temp_y":"Desired Max Temperature"})

# Add Desired Wind Speed column
wind_cond = cities_df[cities_df["Wind Speed"] < 15]
second_combined_cond = pd.merge(first_combined_cond, wind_cond, how='outer', on='City')
second_combined_cond = second_combined_cond.rename(columns={"Wind Speed_y":"Desired Wind Speed"})

# Add Desired Humidity column
humidity_cond = cities_df[cities_df["Humidity"] < 60]
third_combined_cond = pd.merge(second_combined_cond, humidity_cond, how='outer', on='City')
third_combined_cond = third_combined_cond.rename(columns={"Humidity_y":"Desired Humidity"})

#Create DataFrame for just desired conditions
desired_conditions = third_combined_cond[['City', 'Desired Max Temperature', 'Desired Wind Speed', 'Desired Humidity']]

#Drop any null values(conditions that did not meet the requirements)
desired_conditions = desired_conditions.dropna(how='any')
desired_conditions

,City,Desired Max Temperature,Desired Wind Speed,Desired Humidity,Desired Humidity
13,constitucion,84.20,16.11,4.0,4.0
14,albany,82.40,12.75,45.0,45.0
25,colonelganj,94.20,5.08,14.0,14.0
37,mayor pablo lagerenza,91.86,14.90,33.0,33.0
45,port elizabeth,93.00,12.75,46.0,46.0
...,...,...,...,...,...
529,hakkari,69.80,5.82,46.0,46.0
530,marzuq,79.44,6.02,36.0,36.0
533,matam,102.12,11.61,14.0,14.0
539,yabrud,80.60,8.05,32.0,32.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [91]:
hotel_df = desired_conditions

# Add "hotel name" column
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Desired Max Temperature,Desired Wind Speed,Desired Humidity,Desired Humidity,Hotel Name
13,constitucion,84.20,16.11,4.0,4.0,
14,albany,82.40,12.75,45.0,45.0,
25,colonelganj,94.20,5.08,14.0,14.0,
37,mayor pablo lagerenza,91.86,14.90,33.0,33.0,
45,port elizabeth,93.00,12.75,46.0,46.0,


In [92]:
# Create a map and set markers
marker_locations = hotel_df[['City', 'Hotel Name']]

# Create a marker_layer and fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
fig.add_layer(markers)
fig

NameError: name 'poverty_rate' is not defined

In [89]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
